
# The below code does the following:
1. Reads through all files one by one, looking for occurrences of float numbers.

2. Looks for parameter names in the starting index of every line cross referenced by data from 'X1.json', and if found, appends that line to the data

3. To look for latest data for a parameter: it considers the last value in the string as the latest parameter. This is an implicit assumption I am making because dates appear to be arranged chronologically. Therefore by default, the last value will be the latest one.

4. Lastly, I have filtered data and added it to a dictionary as required.

5. Helper Functions I used:
  a) Time pattern matching
  b) Date pattern matching
  c) float detection
  d) file reading
  e) searching in json file for reading through X1.json

# Some things I have not been able to accomplish due to the shortage of time:


1. Correct for OCR errors - I had planned on utilising Levenshtein Distance to correct for OCR errors.

2. Unable to extract the 'unit' key for the dicitonary to non uniformity of unit values. A potential approach: A dictionary listing all kinds of units would have been helpful in extracting the unit keys.



In [154]:
import re
from datetime import datetime

def convert_date_format(text):
    # Define the regex pattern to match dates in the format "30 Nov 18"
    date_pattern = r'(\d{1,2})\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+(\d{2})'

    # Find all matches of the date pattern in the text
    matches = re.findall(date_pattern, text)

    converted_dates = []
    for match in matches:
        # Convert the matched date into Python datetime format
        day, month, year = match
        date_str = f"{day} {month} {year}"
        converted_date = datetime.strptime(date_str, '%d %b %y').date()
        converted_dates.append(converted_date.strftime("%Y-%m-%d"))

    return converted_dates



In [155]:
import re
from datetime import datetime

def convert_time_format(text):

    new_text = str(text.split(" "))
    time_pattern = r'\b(2[0-3]|[01]?[0-9]):([0-5]?[0-9])\b'
    matches = re.findall(time_pattern, new_text)
    converted_times = []
    for match in matches:
        hour, minute = map(int, match)
        time_str = f"{hour}:{minute}"
        converted_time = datetime.strptime(time_str, '%H:%M').time()
        converted_times.append(converted_time.strftime("%H:%M"))

    return converted_times



In [156]:
def read_data_from_files(file_paths):
    all_data = ""  # Variable to store all data from files

    for file_path in file_paths:
        try:
            with open(file_path, 'r') as file:
                data = file.read()
                all_data += data + "\n"  # Append data from file to the variable
        except FileNotFoundError:
            print(f"File '{file_path}' not found. Skipping...")

    return all_data



In [157]:
def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [158]:
def read_data_from_files(file_paths):
    all_data = ""  # Variable to store all data from files

    for file_path in file_paths:
        try:
            with open(file_path, 'r') as file:
                data = file.read()
                all_data += data + "\n"  # Append data from file to the variable
        except FileNotFoundError:
            print(f"File '{file_path}' not found. Skipping...")

    return all_data

file_paths = ["/content/02b832e1-66cc-4f35-8b52-abf41cd821b2.txt", "/content/0ab9800e-bc9a-4388-aaa2-d4fc05e7d111.txt", "/content/0b8706dc-c9af-4c6b-887d-2f85b5a511e7.txt","/content/0c59298d-4205-4f6a-8bc9-c078123da03a.txt","/content/0c848136-de54-49eb-a3c4-b04dda11ef42.txt","/content/0ca602c0-d93a-4ae2-b91f-532db7e174ae.txt","/content/0deee5f5-f9d3-4712-8d2e-c5f7cd9895dc.txt","/content/0ea75106-2a9d-4adc-9dad-33256e7d9aad.txt","/content/0ecb52fd-138f-4566-bdef-06fabdd7019d.txt"]  # Replace with actual file paths
all_data = read_data_from_files(file_paths)


import json

def search_in_json(file_path, search_string):
    # Load the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Iterate through the structure
    for item in data:
        if search_string == item["Abbreviation"]:
            return True
        if search_string in item["Synonyms"]:
            return True

    return False


flag = False
def filter_lines_with_same_occurrences(text):
    lines = text.strip().split('\n')
    considered_data = []
    num_occurrences = None
    index = 0
    for line in lines:
        flag = False;
        if "Request Number" in line:
            num_occurrences = sum(is_float(char) for char in line.split(" "))

        elif "Time Collected" in line:
            num_occurrences = sum(len(convert_time_format(char)) for char in line.split(" "))
        elif any(convert_date_format(char) != [] for char in line.split(" ")):
            for char in line.split(" "): print(convert_date_format(char))
            num_occurrences = sum(len(convert_date_format(char)) for char in line.split(" "))
        else:
            num_occurrences = sum(is_float(char) for char in line.split(" "))

        if num_occurrences is not None and num_occurrences!= 0 and sum(is_float(char) for char in line.split(" ")) == num_occurrences:
            index = 0
            for char in line.split(" "):
              if(not(search_in_json("X1.json", char))):
                  flag = flag + False
              else:
                if(index == 0):
                  flag = flag + True

              index +=1
            if(flag != False and flag!= 0):
              considered_data.append(line)
        elif num_occurrences is not None and num_occurrences!= 0 and sum(len(convert_time_format(char)) for char in line.split(" ")) == num_occurrences:
            index = 0
            for char in line.split(" "):
              if(not(search_in_json("X1.json", char))):
                  flag = flag + False
              else:
                if(index == 0):
                  flag = flag + True
            if(flag != False and flag!= 0):
                considered_data.append(line)
        elif num_occurrences is not None and num_occurrences!= 0 and sum(len(convert_date_format(char)) for char in line.split(" ")) == num_occurrences:
            index = 0
            for char in line.split(" "):
              if(not(search_in_json("X1.json", char))):
                  flag = flag + False
              else:
                if(index == 0):
                  flag = flag + True
            if(flag != False and flag!= 0):
              considered_data.append(line)



    return '\n'.join(considered_data)

considered_data = filter_lines_with_same_occurrences(all_data)

# Print the filtered data
print(considered_data)


TSH (0.5-4.0) miIU/L 0.73 0.82 0.64 0.46
FT4 (10-20) pmol/L 15 13 13
FT3 (3.5-6.0) pmol/L 4.8
Iron (10-30) umol/L 40 33 21 27
Ferritin (30-400) ug/L 63 110 115 44
Anti-Thyroglobulin Abs (Immulite) 110 TU/mL (< 41)
CEA (Bayer) (< 2.6) ug/L 4.2 3.4 4.1 3.4
CA125 (< 35) U/mL 17 11 15 13
B12 (301-740) pmol/L 252 408 860 537
Na (135-145) mmol/L 140 142 138 138
K (3.6-5.4) mmol/L 4.5 4.0 3.8 4.2
Cl (95-110) mmol/L 106 108 101 106
HCO3 (22-32) mmol/L 24 24 26 25
Urea (3.0-10.0) mmol/L 6.5 5.7 2.3 6.7
eGFR mL/min/1.73m*2 88 83 > 90 81
Urate (0.14-0.36) mmol/L 0.28 0.14 0.21
AST (< 35) U/L 25 26 26 19
ALT (< 30) U/L 20 17 17 13
GGT (< 35) U/L 11 12 12 12
Ca (2.10-2.60) mmol/L 2.39 2.43 2.41 2.41
PO4 (0.75-1.50) mmol/L 1.00 1.95 1.01 1.24
CPK (< 211) U/L 160 64 68
Amylase (< 121) U/L 55 57 73
Mg (0.70-1.10) mmol/L 0.82 0.86 0.90
eGFR values between 60 and 89 mL/min/1.73m2 should be interpreted with
ESR (< 30) mm/hr 2 2 5 2
Hb (115-165) g/L 141 125 141 130
Hct (0.34-0.47) 0.43 0.37 0.41 0.40
MCV 

In [159]:
def filter_data(line):
  new_line = ""
  dict1={}
  arr =[]
  latest_val_added = False
  for char in line.split(" "):

    if(search_in_json("X1.json", char)):

      new_line = new_line + 'parameter = ' + char
      dict1['parameter'] = char

    elif(is_float(char)) and not latest_val_added:
      for minichar in line.split(" "):
        if(is_float(minichar)):
          latest_val = minichar
      new_line = new_line + 'value = ' + latest_val
      dict1['value'] = latest_val
      latest_val_added = True

    else:
      a=0

  arr.append(dict1)
  return dict1
array1 = []
for line in considered_data.split('\n'):
  array1.append(filter_data(line))

print(array1)


[{'parameter': 'TSH', 'value': '0.46'}, {'parameter': 'FT4', 'value': '13'}, {'parameter': 'FT3', 'value': '4.8'}, {'parameter': 'Iron', 'value': '27'}, {'parameter': 'Ferritin', 'value': '44'}, {'parameter': 'Anti-Thyroglobulin', 'value': '110'}, {'parameter': 'CEA', 'value': '3.4'}, {'parameter': 'CA125', 'value': '13'}, {'parameter': 'B12', 'value': '537'}, {'parameter': 'Na', 'value': '138'}, {'parameter': 'K', 'value': '4.2'}, {'parameter': 'Cl', 'value': '106'}, {'parameter': 'HCO3', 'value': '25'}, {'parameter': 'Urea', 'value': '6.7'}, {'parameter': 'eGFR', 'value': '81'}, {'parameter': 'Urate', 'value': '0.21'}, {'parameter': 'AST', 'value': '19'}, {'parameter': 'ALT', 'value': '13'}, {'parameter': 'GGT', 'value': '12'}, {'parameter': 'Ca', 'value': '2.41'}, {'parameter': 'PO4', 'value': '1.24'}, {'parameter': 'CPK', 'value': '68'}, {'parameter': 'Amylase', 'value': '73'}, {'parameter': 'Mg', 'value': '0.90'}, {'parameter': 'eGFR', 'value': '89'}, {'parameter': 'ESR', 'value':